##NOTES TO SELF
The ECDF reference region thing works.
On reduced data, if my calculations are right, a voxelwise ECDF shouldn't be too time-consuming (1-10 minutes).
Try this out before bothering with vectorizing it. Use map with a lambda function like this ECDF(array)(array)
Afterwards, add the ref sample stuff
And then eventually the inputs and outputs etc

In [2]:
import os
from glob import glob
import numpy as np
import pandas
from nilearn import image, input_data
import nibabel as ni
import scipy.stats as stats
from scipy.io import savemat
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import FeatureAgglomeration
from sklearn.feature_extraction.image import grid_to_graph
import statsmodels.distributions.empirical_distribution as ed

In [4]:
def prepare_PET_data(files_in, atlas, ref = None, msk = None, dimension_reduction = False,
                     ECDF_in = None, output_type = 'py', out_dir = './', out_name = 'PET_data', 
                     save_matrix = False, save_ECDF = False, save_images = False, ref_index = []):
    ''' This is a function that will take several PET images and an atlas and will
    return a subject X region matrix. If specified, the function will also calculate 
    probabilities (via ECDF) either voxelwise, or using a specified reference region
    
    files_in = input can either be 
        - a path to a directory full of (only) nifti images OR
        - a "search string" using wildcards
        - a list of subject paths OR
        - a subject X image matrix
        
    altas = a path to a labeled regional atlas in the same space as the PET data
    
    ref = multiple options:
        - If None, no probabilities will be calculated, and script will simply extract
        regional PET data using the atlas.
        - If a path to a reference region mask, will calculate voxelwise probabilities
        based on values within the reference region. Mask must be in the same space as 
        as PET data and atlas
        - If a list of integers, will combine these atlas labels with these integers to 
        make reference region 
        - if 'voxelwise', voxelwise (or atom-wise from dimension reduction) probabilities
        will be estimated. In other words, each voxel or atom will use serve as its own
        reference.
        
    msk = A path to a binary mask file in the same space as PET data and atlas. If None,
        mask will be computed as a binary mask of the atlas.
        ** PLEASE NOTE: The mask will be used to mask the reference region! **
    
    dimension_reduction = whether or not to first reduce dimensions of data using
    hierarchical clustering. This results in an initial step that will be very slow, but 
    will may result in an overall speedup for the script, but perhaps only if ref is set 
    to 'voxelwise'.
        - If None, do not perform dimension reduction
        - If integer, the number of atoms (clusters) to reduce to
    
    ECDF_in = If the user wishes to apply an existing ECDF to the PET data instead of
        generating one de novo, that can be done here. This crucial if the user wishes to
        use multiple datasets. Think of it like scaling in machine learning.
        - If None, will generate ECDF de novo.
        - If np.array, will use this array to generate the ECDF.
        - If statsmodel ECDF object, will use this as ECDF
        - If a path, will use the
    
    output_type = type of file to save final subject x region matrix into. multiple options:
        -- 'py' will save matrix into a csv
        -- 'mat' will save matrix into a matfile
    
    out_dir = location to save output files. Defaults to current directory
    
    out_name = the prefix for all output files
    
    save_matrix = Whether to save or return subject x image matrix. Useful if running multiple 
        times, as this matrix can be set as files_in, bypassing the costly data import
        -- if 'return', will return subject x image matrix to python environment
        -- if 'save', will write subject x image matrix to file. 
        -- if None, matrix will not be stored
    
    save_ECDF = whether to save the ECDF used to create the probabilities. This is crucial if 
        using multiple datasets. The resulting output can be used as input for the ECDF argument.
        -- if 'return, will return np.array to python environment
        -- if 'save', will write array to file
        -- if None, array will not be stored
    
    '''
    # Check input arguments
    print('initiating...')
    if output_type != 'py' and output_type != 'mat':
        raise IOError('output_type must be set to py or mat')
    
    
    # Initialize variables
    
    # Load data
    print('loading data...')
    i4d = load_data(files_in) # load PET data
    if save_matrix == 'save':
        otpt = os.path.join(out_dir,'%s_4d_data'%out_name)
        print('saving 4d subject x scan to nifti image: \n',otpt)
        i4d.to_filename(otpt)
    
    # load atlas
    atlas = ni.load(atlas).get_data().astype(int) 
    if atlas.shape != i4d.shape[:3]:
        raise ValueError('atlas dimensions do not match PET data dimensions')
    
    # load reference region
    if type(ref) == str and ref != 'voxelwise': 
        print('looking for reference image...')
        if not os.path.isdir(ref):
            raise IOError('Please enter a valid path for ref, or select a different option for this argument')
        else:
            ref_msk = ni.load(ref).get_data()
            if ref_msk.shape != i4d.shape[:3]:
                raise ValueError('ref region image dimensions do not match PET data dimensions')
    elif type(ref) == list:
        ref_msk = np.zeros_like(atlas)
        for i in ref:
            ref_msk[atlas == i] = 1
    else:
        ref_msk = None
    
    
    # Mask data
    print('masking data...')
    if type(msk) == type(None):
        img_mask = atlas[:,:,:]
        img_mask[img_mask<1] = 0
        img_mask[img_mask>0] = 1
    else:
        img_mask = ni.load(msk).get_data()
        atlas[img_mask < 1] = 0
    
    if type(ref_msk) != type(None):
        ref_msk[img_mask < 1] = 0
    
    mask_tfm = input_data.NiftiMasker(ni.Nifti1Image(img_mask,i4d.affine))
    mi4d = mask_tfm.fit_transform(i4d)
    
    # dimension reduction (IN BETA!)
    if dimension_reduction:
        print('reducing dimensions...')
        shape = img_mask.shape
        connectivity = grid_to_graph(n_x=shape[0], n_y=shape[1],
                                   n_z=shape[2], mask=img_mask)
    # main ECDF calculation
    skip = False
    if ref != 'voxelwise':
        if type(ECDF_in) != type(none): 
            print('generating ECDF...')
            print('using user-supplied data...')
            if type(ECDF_in) == ed.ECDF:
                mi4d_ecdf, ecref = ecdf_simple(mi4d, ECDF_in)
                input_distribution = 'not generated'
            elif type(ECDF_in) == np.ndarray:
                mi4d_ecdf, ecref = ecdf_simple(mi4d, ECDF_in)
                input_distribution = ECDF_in
    #       elif # add later an option for importing an external object 
            else:
                try:
                    mi4d_ecdf, ecref = ecdf_simple(mi4d, ECDF_in)
                    print('Could not understand ECDF input, but ECDF successful')
                    input_distribution = 'not generated'
                except:
                    raise IOError(
                            'Invalid argument for ECDF in. Please enter an ndarray, an ECDF object, or a valid path')
        else:
            if type(ref_msk) != type(None):
                print('generating ECDF...')
                ref_tfm = input_data.NiftiMasker(ni.Nifti1Image(ref_msk,i4d.affine))
                refz = ref_tfm.fit_transform(i4d)
                mi4d_ecdf, ecref = ecdf_simple(mi4d, refz)
                input_distribution = refz.flat
            else:
                print('skipping ECDF...')
                skip = True
    
    else:
        print('generating voxelwise ECDF...')
        mi4d_ecdf, ECDF_array = ecdf_voxelwise(mi4d, ref_index, save_ECDF)
        input_distribution = 'not generated'
        
    if not skip:
#       if save_ECDF:
#           create an array and somehow write it to a file
        
    # transform back to image-space
        print('transforming back into image space')
        f_images = mask_tfm.reverse_transform(mi4d_ecdf)
    
    else:
        if type(ECDF)
        print('transforming back into image space')
        f_images = mask_tfm.reverse_transform(mi4d)
    
    # generate output matrix
    print('generating final subject x region matrix')
    f_mat = generate_matrix_from_atlas(f_images, atlas)
    
    # compile (and save) outputs
    print('preparing outputs')
    output = {}
    if output_type = 'py':
        f_mat.to_csv(os.path.join(out_dir, '%s_roi_data.csv'%out_name))
        output.update({'roi_matrix': f_mat})
    else:
        output.update({'roi_matrix': fmat.values})
        output.update({'roi_matrix_columns': fmat.columns})
    if save_matrix == 'return':
        output.update({'4d_image_matrix': i4d})
    if save_ECDF == 'return':
        if output_type == 'py':
            output.update({'ECDF_function': ECDF_array})
        else:
            output.update({'input_distribution': input_distribution})
    
def load_data(files_in):
    
    fail = False
    
    if type(files_in) == str:
        if os.path.isdir(files_in):
            print('It seems you passed a directory')
            search = os.path.join(files_in,'*')
            num_f = len(glob(search))
            if num_f == 0:
                raise IOError('specified directory did not contain any files')
            else:
                print('found %s images!'%num_f)
            i4d = image.load_img(search)
        elif '*' in files_in:
            print('It seems you passed a search string')
            num_f = len(glob(files_in))
            if num_f == 0:
                raise IOError('specified search string did not result in any files')
            else:
                print('found %s images'%num_f)
            i4d = image.load_img(files_in)
        else:
            fail = True
    elif type(files_in) == list:
        print('processing %s subjects'%len(files_in))
        i4d = ni.concat_images(files_in)
    elif type(files_in) == ni.nifti1.Nifti1Image:
        print('processing %s subjects'%files_in.shape[-1])
        i4d = files_in
    else:
        fail = True
        
    if fail:
        print('files_in not recognized.', 
                    'Please enter a search string, valid directory, list of subjects, or matrix')
        raise ValueError('I do not recognize the files_in input.')
    
    return i4d

def dim_reduction(mi4d, connectivity, dimension_reduction):
    ward = FeatureAgglomeration(n_clusters=dimension_reduction/2,
            connectivity=connectivity, linkage='ward', memory='nilearn_cache')
    ward.fit(mi4d)
    ward = FeatureAgglomeration(n_clusters=dimension_reduction,
            connectivity=connectivity, linkage='ward', memory='nilearn_cache')
    ward.fit(mi4d)                                                         
    mi4d = ward.transform(mi4d)

    return mi4d

def ecdf_simple(mi4d, refz):

    if type(refz) == ed.ECDF:
        ecref = refz
    else:
        if len(refz.shape) > 1:
            ecref = ed.ECDF(refz.flat)
        else:
            ecref = ed.ECDF(refz)
    print('transforming images...')
    mi4d_ecdf = ecref(mi4d.flat).reshape(mi4d.shape[0],mi4d.shape[1])

    return mi4d_ecdf, ecref   

def ecdf_voxelwise(mi4d, ref_index, save_ECDF):
    
    X,y = mi4d.shape

    if len(ref_index) == 0:
        if not save_ECDF:
            mi4d_ecdf = np.array([ed.ECDF(mi4d[:,x])(mi4d[:,x]) for x in range(y)]).reshape(X,y)
            ECDF_array = None
        else:
            ECDF_array = np.array([ed.ECDF(mi4d[:,x]) for x in range(y)]).reshape(X,y)
            print('transforming data...')
            mi4d_ecdf = np.array([ECDF_matrix[x](mi4d[:,x]) for x in range(y)]
                                     ).reshape(X,y)
    else:
        good_ind = [x for x in list(range(X)) if x not in ref_index]
        if not save_ECDF:    
            mi4d_ecdf = np.array([ed.ECDF(mi4d[ref_index,x])(mi4d[good_ind,x]) for x in range(y)]
                                ).reshape(X,y)
            ECDF_array = None
        else:
            ECDF_array = [ed.ECDF(mi4d[ref_index,x]) for x in range(y)]
            print('transforming data...')
            mi4d_ecdf = ecdf_voxelwise = np.array([ECDF_matrix[x](mi4d[good_ind,x]) for x in range(y)]
                                     ).reshape(X,y)
    
    return mi4d_ecdf, ECDF_array

def generate_matrix_from_atlas(files_in, atlas):
    
    atlas = atlas.astype(int)
    f_mat = pandas.DataFrame(index = range(files_in.shape[-1]),
                             columns = ['roi_%s'%x for x in np.unique(atlas) if x != 0])
    tot = np.bincount(atlas.flat)
    for sub in range(files_in.shape[-1]):
        mtx = f_mat[:,:,:,sub]
        sums = np.bincount(atlas.flat, weights = mtx.flat)
        rois = (sums/tot)[1:]
        f_mat.loc[fmat.index[sub]] = rois
        
    return f_mat



In [5]:
scans = sorted(glob('/Users/jakevogel/Science/tau/nan_snorm_*002*'))
i4d = load_data(scans)

processing 6 subjects


In [6]:
atlas = '/Users/jakevogel/Science/tau/dkt_atlas_1mm.nii.gz'
ref = [79,80]
msk = '/Users/jakevogel/Science/tau/thr_ADNI_GM_mask_1mm.nii.gz'

# load atlas
atlas = ni.load(atlas).get_data().astype(int) 
if atlas.shape != i4d.shape[:3]:
    raise ValueError('atlas dimensions do not match PET data dimensions')

# load reference region
if type(ref) == str and ref != 'voxelwise': 
    print('looking for reference image...')
    if not os.path.isdir(ref):
        raise IOError('Please enter a valid path for ref, or select a different option for this argument')
    else:
        ref_msk = ni.load(ref).get_data()
        if ref_msk.shape != i4d.shape[:3]:
            raise ValueError('ref region image dimensions do not match PET data dimensions')
elif type(ref) == list:
    ref_msk = np.zeros_like(atlas)
    for i in ref:
        ref_msk[atlas == i] = 1
else:
    ref_msk = None


# Mask data
print('masking data...')
if msk == None:
    img_mask = atlas[:,:,:]
    img_mask[img_mask<1] = 0
    img_mask[img_mask>0] = 1
else:
    img_mask = ni.load(msk).get_data()
    atlas[img_mask < 1] = 0

if type(ref_msk) != type(None):
    ref_msk[img_mask < 1] = 0

mask_tfm = input_data.NiftiMasker(ni.Nifti1Image(img_mask,i4d.affine))
mi4d = mask_tfm.fit_transform(i4d)

masking data...


In [75]:
ECDF_in = None
skip = False
if ref != 'voxelwise':
    if ECDF_in: 
        print('generating ECDF...')
        print('using user-supplied data...')
        if type(ECDF_in) == ed.ECDF:
            mi4d_ecdf, ecref = ecdf_simple(mi4d, ECDF_in)
        elif type(ECDF_in) == np.ndarray:
            mi4d_ecdf, ecref = ecdf_simple(mi4d, ECDF_in)
#       elif # add later an option for importing an external object 
        else:
            try:
                mi4d_ecdf, ecref = ecdf_simple(mi4d, ECDF_in)
            except:
                raise IOError(
                        'Invalid argument for ECDF in. Please enter an ndarray, an ECDF object, or a valid path')
    else:
        if type(ref_msk) != type(None):
            print('generating ECDF...')
            ref_tfm = input_data.NiftiMasker(ni.Nifti1Image(ref_msk,i4d.affine))
            refz = ref_tfm.fit_transform(i4d)
            mi4d_ecdf, ecref = ecdf_simple(mi4d, refz)
        else:
            print('skipping ECDF...')
            skip = True

generating ECDF...
transforming images...


In [7]:
X,y = mi4d.shape
ecdf_voxelwise = np.array(map(lambda x: ed.ECDF(mi4d[:,x]),range(y))).reshape(X,y)

ValueError: total size of new array must be unchanged

In [9]:
X,y = mi4d.shape
ecdf_voxelwise = [ed.ECDF(mi4d[:,x])(mi4d[:,x]) for x in range(y)]

In [ ]:
tst = np.array(ecdf_voxelwise).reshape(X,y)

In [14]:
jnk_ind = [2,3]
mi4d[jnk_ind,0]

array([ 1.23515558,  0.81288999])

In [24]:
jnk1 = list(range(X)[2:])
print(jnk1)
good_ind = [x for x in range(X) if x not in jnk1]
good_ind

[2, 3, 4, 5]


[0, 1]

In [25]:
ecdf_voxelwise = [ed.ECDF(mi4d[jnk1,x])(mi4d[good_ind,x]) for x in range(y)]

In [27]:
ECDF_matrix = [ed.ECDF(mi4d[:,x]) for x in range(y)]

In [31]:
ecdf_voxelwise = [ECDF_matrix[x](mi4d[good_ind,x]) for x in range(y)]

In [80]:
mi4d_ecdf.shape

(6, 939852)

In [ ]:
ecdf_voxelwise = [ed.ECDF(mi4d[:,x])(mi4d[:,x]) for x in range(y)]

In [86]:
nimg = mask_tfm.inverse_transform(mi4d_ecdf[:2])

In [88]:
nimg.to_filename('/Users/jakevogel/Science/tau/ecdf_test')

In [35]:
atlas_tfm = input_data.NiftiLabelsMasker(atlas, mask_img=img_mask, resampling_target=None)

In [38]:
tst_img = mask_tfm.inverse_transform(tst)

In [40]:
type(tst_img)

nibabel.nifti1.Nifti1Image

In [39]:
tst_atl = atlas_tfm.fit_transform(tst_img)

TypeError: Data given cannot be loaded because it is not compatible with nibabel format:
0

In [ ]:
mi4d.shape

In [67]:
img_mask = ni.load(msk).get_data()
img_mask.shape

(182, 218, 182)

In [64]:
atlas.shape

(182, 218, 182)

In [47]:
len(img_mask[img_mask<1])

4169090

In [14]:
scans = sorted(glob('/home/users/jvogel/Science/datasets/ADNI_AV1451_template_space/*_S_*/*'))
len(scans)

91

In [17]:
i4d = ni.concat_images(scans)

In [21]:
i4d.shape[:3]

(121, 145, 121)

In [19]:
type(i4d)

nibabel.nifti1.Nifti1Image

In [24]:
atlas = ni.load('/home/users/jvogel/Science/templates/atlases/HarvardOxford-sub-maxprob-thr0-1p5mm.nii.gz').get_data()
atlas.shape

(121, 145, 121)

In [25]:
atlas = atlas.astype(int)

In [35]:
ref = np.zeros_like(atlas)
for i in jnk:
    ref[atlas==i] = 1
len(ref[ref > 0])

4464

In [36]:
img_mask = atlas[:,:,:]
img_mask[img_mask<1] = 0
img_mask[img_mask>0] = 1

In [50]:
mask_tfm = input_data.NiftiMasker(ni.Nifti1Image(img_mask,i4d.affine))

In [51]:
mi4d = mask_tfm.fit_transform(i4d)
mi4d.shape

(91, 732986)

In [41]:
type(img_mask)

numpy.ndarray

In [9]:
jnk = np.random.randn(100)

In [13]:
ejnk = ECDF(abs(jnk))

In [16]:
ejnk([3, 2, 1.5, 2.3])

array([ 1.  ,  0.97,  0.87,  0.99])

In [20]:
ejnk = ECDF([1,1.2,1,1.2,1,1.2,1.5,1,1.2,1,1.2,1.5,1,1.2,1.8,2,3])


In [30]:
jnk([1,1.2,1.5,1.8,2,3])

array([ 0.33333333,  0.33333333,  0.33333333,  0.33333333,  0.66666667,
        0.83333333])

In [32]:
jnk = np.random.randn(91)

In [33]:
%%timeit
r = ECDF(jnk)

The slowest run took 7.58 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 78.9 µs per loop


In [35]:
r = ECDF(jnk)

In [36]:
%%timeit
r(jnk)

The slowest run took 10.61 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.22 µs per loop


In [37]:
78.9 + 7.22

86.12

In [40]:
(86.12 * 1000000) / 1000000

86.12

In [39]:
602/60

10.033333333333333

In [79]:
ECDF(jnk)(jnk)

array([ 0.46153846,  0.85714286,  0.86813187,  0.53846154,  0.61538462,
        0.89010989,  0.37362637,  0.69230769,  0.35164835,  0.24175824,
        0.32967033,  0.95604396,  0.16483516,  0.1978022 ,  0.93406593,
        0.92307692,  0.08791209,  0.05494505,  0.57142857,  0.91208791,
        0.41758242,  0.6043956 ,  0.43956044,  0.83516484,  0.0989011 ,
        0.48351648,  0.45054945,  0.03296703,  0.30769231,  0.9010989 ,
        0.12087912,  0.47252747,  0.42857143,  0.59340659,  0.98901099,
        0.3956044 ,  0.54945055,  0.49450549,  0.67032967,  0.73626374,
        0.01098901,  0.56043956,  0.8021978 ,  0.38461538,  0.96703297,
        1.        ,  0.04395604,  0.81318681,  0.17582418,  0.28571429,
        0.7032967 ,  0.10989011,  0.15384615,  0.18681319,  0.20879121,
        0.02197802,  0.25274725,  0.34065934,  0.87912088,  0.97802198,
        0.07692308,  0.27472527,  0.2967033 ,  0.79120879,  0.58241758,
        0.50549451,  0.74725275,  0.71428571,  0.13186813,  0.76